In [1]:
import numpy as np
import pandas as pd

In [2]:
nba= pd.read_csv('nba.csv', low_memory= False)

# Original Data

In [3]:
pd.set_option('display.max_rows', None)
nba.head(3).T

,0,1,2
Unnamed: 0,0,1,2
game_id,202202170BRK,202202170BRK,202202170BRK
game_date,2022-02-17,2022-02-17,2022-02-17
OT,0,0,0
H_A,A,A,A
Team_Abbrev,WAS,WAS,WAS
Team_Score,117,117,117
Team_pace,94.5,94.5,94.5
Team_efg_pct,0.627,0.627,0.627
Team_tov_pct,13.5,13.5,13.5


In [4]:
##Entity- table
##Attribute- column
##Primary Key- one or more columns in a table that a unique in atable that are unique on every row


## First Normal Form

In [5]:
nba1= nba.drop(['Inactives'], axis=1)
nba1.head(3).T

,0,1,2
Unnamed: 0,0,1,2
game_id,202202170BRK,202202170BRK,202202170BRK
game_date,2022-02-17,2022-02-17,2022-02-17
OT,0,0,0
H_A,A,A,A
Team_Abbrev,WAS,WAS,WAS
Team_Score,117,117,117
Team_pace,94.5,94.5,94.5
Team_efg_pct,0.627,0.627,0.627
Team_tov_pct,13.5,13.5,13.5


In [6]:
##primary dependent depends on stuffs outside the primary key
##The key(First normal form), the whole key (Second normal form), nothing but the key (Third normal form).

In [7]:
##Primay key for this data is game id and player id

# Second Normal Form

In [8]:
players = nba1[['player_id', 'player']].drop_duplicates()
#players

In [9]:
nba2 = nba1.drop(['player'], axis=1) 

In [10]:
games = nba1[['game_id', 'game_date', 'OT', 'season']].drop_duplicates()
#games

In [11]:
nba = nba.drop(['game_date', 'OT', 'season'], axis=1)
#nba.head(3).T

In [12]:
# nba = nba.drop(['game_date', 'OT', 'season'], axis=1, errors='ignore')
# nba.head(3).T

# Third Normal Form

In [13]:
nba.columns

Index(['Unnamed: 0', 'game_id', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Inactives', 'Opponent_Abbrev',
       'Opponent_Score', 'Opponent_pace', 'Opponent_efg_pct',
       'Opponent_tov_pct', 'Opponent_orb_pct', 'Opponent_ft_rate',
       'Opponent_off_rtg', 'player', 'player_id', 'starter', 'mp', 'fg', 'fga',
       'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb',
       'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus',
       'did_not_play', 'is_inactive', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct',
       'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct',
       'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'bpm',
       'minutes', 'double_double', 'triple_double', 'DKP', 'FDP', 'SDP',
       'DKP_per_minute', 'FDP_per_minute', 'SDP_per_minute', 'pf_per_minute',
       'ts', 'last_60_minutes_per_game_starting',
     

In [14]:
team_game = nba[['game_id','Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev']].drop_duplicates

In [15]:
#team_game.head(4).T

In [16]:
# pd.set_option('display.max_columns', 80)
# nba.query("game_id"=='202202170')

In [17]:
player_game = nba.drop(['H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Inactives', 'Opponent_Abbrev',
       'Opponent_Score', 'Opponent_pace', 'Opponent_efg_pct',
       'Opponent_tov_pct', 'Opponent_orb_pct', 'Opponent_ft_rate',
       'Opponent_off_rtg'], axis=1)

In [18]:
#player_game